In [ ]:
%tensorboard --logdir=logdir 

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Simple Feed-forward GAN
class Generator(nn.Module):
    def __init__(self, z_dim=20, out_dim=784):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, out_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x)

class Discriminator(nn.Module):
    def __init__(self, in_dim=784):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Data Preparation
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

# Initialize Networks
z_dim = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G = Generator(z_dim=z_dim).to(device)
D = Discriminator().to(device)

# Initialize Optimizers
lr = 0.0002
G_optimizer = Adam(G.parameters(), lr=lr)
D_optimizer = Adam(D.parameters(), lr=lr)

# Initialize Loss Function
criterion = nn.BCELoss()

# TensorBoard Summary Writer
writer = SummaryWriter()

# Training Loop
num_epochs = 100
fixed_noise = torch.randn(100, z_dim).to(device)
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        real_images = real_images.view(real_images.size(0), -1).to(device)
        batch_size = real_images.size(0)

        # Train Discriminator
        D_real = D(real_images)
        D_real_loss = criterion(D_real, torch.ones(batch_size, 1).to(device))

        z = torch.randn(batch_size, z_dim).to(device)
        fake_images = G(z)
        D_fake = D(fake_images)
        D_fake_loss = criterion(D_fake, torch.zeros(batch_size, 1).to(device))

        D_loss = D_real_loss + D_fake_loss

        D_optimizer.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train Generator
        z = torch.randn(batch_size, z_dim).to(device)
        fake_images = G(z)
        D_fake = D(fake_images)
        G_loss = criterion(D_fake, torch.ones(batch_size, 1).to(device))

        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        # Log Losses
        if i % 50 == 0:
            writer.add_scalar('Loss/Discriminator', D_loss.item(), epoch * len(dataloader) + i)
            writer.add_scalar('Loss/Generator', G_loss.item(), epoch * len(dataloader) + i)

    # Log Generated Images
    if epoch % 5 == 0:
        with torch.no_grad():
            fake_images = G(fixed_noise)
            img_grid = make_grid(fake_images.view(-1, 1, 28, 28), nrow=10, normalize=True)
            writer.add_image('Generated Images', img_grid, global_step=epoch)

writer.close()


KeyboardInterrupt: 

In [2]:
writer.close()

In [4]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.0000000001,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

acc,▂▁▆▄▆█▇▇
loss,█▃▄▂▁▁▁▁
acc,0.822
loss,0.15463
